In [2]:
pip install numpy

  Using cached numpy-2.0.2-cp39-cp39-win_amd64.whl.metadata (59 kB)
Using cached numpy-2.0.2-cp39-cp39-win_amd64.whl (15.9 MB)
Note: you may need to restart the kernel to use updated packages.


In [14]:
import numpy as np
import json

In [5]:
pip install scikit-learn

  Using cached scikit_learn-1.6.1-cp39-cp39-win_amd64.whl.metadata (15 kB)
  Using cached scipy-1.13.1-cp39-cp39-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.6.1-cp39-cp39-win_amd64.whl (11.2 MB)
Using cached joblib-1.5.1-py3-none-any.whl (307 kB)
Using cached scipy-1.13.1-cp39-cp39-win_amd64.whl (46.2 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)

   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- -------------------------

In [6]:
from sklearn.model_selection import train_test_split

In [17]:
def generate_leaky_data(n_samples=1000, random_state=42):
    """Generates synthetic data with injected leaks.
    Returns:
        X_train, X_test, y_train, y_test, X_leaky, leak_flags
    """
    np.random.seed(random_state)
    X = np.random.normal(0, 1, (n_samples, 5))
    y = (X[:, 0] + X[:, 1] + 0.1 * np.random.normal(0, 1, n_samples) > 0).astype(int)
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)
    
    # Track which leaks are injected
    leak_flags = {
        "overlap": False,
        "multitest": False,
        "preprocessing": False,
        "target": False
    }
    
    # Inject Overlap Leakage (5% of test data added to train)
    overlap_idx = np.random.choice(len(X_test), int(0.05 * len(X_test)), replace=False)
    X_train = np.vstack([X_train, X_test[overlap_idx]])
    y_train = np.concatenate([y_train, y_test[overlap_idx]])
    leak_flags["overlap"] = True
    
    # Создаём X_leaky на основе обновлённого X_train + X_test (все данные)
    X_all = np.vstack([X_train, X_test])
    y_all = np.concatenate([y_train, y_test])
    
    # Inject Target Leakage (add a feature correlated with y)
    leaky_feature = np.random.choice([0, 1], size=len(y_all), p=[0.4, 0.6]) * y_all
    X_leaky = np.column_stack([X_all, leaky_feature])
    leak_flags["target"] = True
    
    # Preprocessing leakage флаг
    leak_flags["preprocessing"] = True
    
    return X_train, X_test, y_train, y_test, X_leaky, leak_flags

In [18]:
if __name__ == "__main__":
    X_train, X_test, y_train, y_test, X_leaky, leak_flags = generate_leaky_data()
    data = {
        "X_train": X_train.tolist(),
        "X_test": X_test.tolist(),
        "y_train": y_train.tolist(),
        "y_test": y_test.tolist(),
        "X_leaky": X_leaky.tolist(),
        "leak_flags": leak_flags
    }
    with open("synthetic_data.json", "w") as f:
        json.dump(data, f)
    print("Synthetic data saved to 'synthetic_data.json'")


Synthetic data saved to 'synthetic_data.json'
